<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/Bloxorz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import copy
import time
import os
import random

In [3]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [5]:
# Create a dictionary to save memory later
move = {0 : "NONE", 1 : "UP", 2 : "DOWN", 3 : "LEFT", 4 : "RIGHT"}
state = {0 : "STANDING", 1 : "LYING_HORIZONTALLY", 2 : "LYING_VERTICALLY", 3 : "SPLIT"}

In [6]:
# Position of the block in the map
class Position:
  def __init__(self, i1, j1, i2, j2, parent = None, move_from_parent = None):
    self.i1 = i1
    self.j1 = j1
    self.i2 = i2
    self.j2 = j2
    self.parent = parent
    self.move_from_parent = move_from_parent

  def print(self):
    print("(", self.i1, self.j1, ")",  "(", self.i2, self.j2, ")")

  def get_state(self):
    if self.i1 == self.i2 and self.j1 == self.j2: return 0
    elif (self.j1 == self.j2+1 or self.j1 == self.j2-1) and self.i1 == self.i2: return 1
    elif (self.i1 == self.i2+1 or self.i1 == self.i2-1) and self.j1 == self.j2: return 2
    else: return 3
  
  def move(self, instruction):
    state = self.get_state()
    if instruction == 0: return copy.copy(self)
    elif instruction == 1:
      if state == 0: return Position(self.i1-2, self.j1, self.i2-1, self.j2, self, 1)
      elif state == 1: return Position(self.i1-1, self.j1, self.i2-1, self.j2, self, 1)
      elif state == 2: return Position(min(self.i1, self.i2)-1, self.j1, min(self.i1, self.i2)-1, self.j2, self, 1)
    elif instruction == 2: 
      if state == 0: return Position(self.i1+2, self.j1, self.i2+1, self.j2, self, 2)
      elif state == 1: return Position(self.i1+1, self.j1, self.i2+1, self.j2, self, 2)
      elif state == 2: return Position(max(self.i1, self.i2)+1, self.j1, max(self.i1, self.i2)+1, self.j2, self, 2)
    elif instruction == 3:
      if state == 0: return Position(self.i1, self.j1-2, self.i2, self.j2-1, self, 3)
      elif state == 1: return Position(self.i1, min(self.j1, self.j2)-1, self.i2, min(self.j1, self.j2)-1, self, 3)
      elif state == 2: return Position(self.i1, self.j1-1, self.i2, self.j2-1, self, 3)
    elif instruction == 4:
      if state == 0: return Position(self.i1, self.j1+2, self.i2, self.j2+1, self, 4)
      elif state == 1: return Position(self.i1, max(self.j1, self.j2)+1, self.i2, max(self.j1, self.j2)+1, self, 4)
      elif state == 2: return Position(self.i1, self.j1+1, self.i2, self.j2+1, self, 4)
  def is_visited(self, visited):
        return ([self.i1, self.j1, self.i2, self.j2] in visited) or ([self.i2, self.j1, self.i1, self.j2] in visited) or ([self.i1, self.j2, self.i2, self.j1] in visited) or ([self.i2, self.j2, self.i1, self.j1] in visited)
  def expand(self):
    return [self.move("UP"), self.move("DOWN"), self.move("LEFT"), self.move("RIGHT")]

In [36]:
class BLOXORZ:
  def __init__(self, map, initial_position):
    self.map = map
    self.initial_position = initial_position
    self.goal = [(index, row.index(10)) for index, row in enumerate(map) if 10 in row][0]

  def is_goal(self, current_position):
    return current_position.get_state() == 0 and self.map[current_position.i1][current_position.j1] == 10

  def is_valid_position(self, current_position):
    if current_position.i1 < 0 or current_position.i2 < 0 or current_position.j1 < 0 or current_position.j2 < 0: return False
    if current_position.i1 >= len(self.map) or current_position.i2 >= len(self.map) or current_position.j1 >= len(self.map[0]) or current_position.j2 >= len(self.map[0]): return False

    if current_position.get_state() == 0: return self.map[current_position.i1][current_position.j1] > 1
    else: return self.map[current_position.i1][current_position.j1] > 0 and self.map[current_position.i2][current_position.j2] > 0
  
  # Estimate cost to goal, calculated my average of Manhattan distances of 2 part of the block
  # Or Chebychev
  def cost_to_goal(self, current_position):
    # return 1/2 * (abs(current_position.i1 - self.goal[0]) + abs(current_position.i2 - self.goal[0]) + abs(current_position.j1 - self.goal[1]) + abs(current_position.j2 - self.goal[1]))
    max1 = max(abs(current_position.i1 - self.goal[0]), abs(current_position.j1 - self.goal[1]))
    max2 = max(abs(current_position.i2 - self.goal[0]), abs(current_position.j2 - self.goal[1]))
    return max(max1, max2)
  def move_process(self, process):
    current = copy.copy(self.initial_position)
    for move in process: 
      current = current.move(move)
      if not self.is_valid_position(current): 
        return -1

    return current


  def DFS(self, stack, visited, loop):
      while len(stack) > 0:
          this_position = stack.pop()
          visited.append([this_position.i1, this_position.j1, this_position.i2, this_position.j2])
          exp_list = this_position.expand()
          for pos in exp_list:
              if self.is_valid_position(pos):
                  if not pos.is_visited(visited):
                      stack.append(pos)
                      # self.draw_map_2D(pos)
                  if self.is_goal(pos): 
                      path = [pos]
                      while path[-1].parent != None: path.append(path[-1].parent)
                      path.reverse()
                      print("Goal reached after", loop, "loops")
                      return path
          loop = loop + 1
      if len(stack) == 0:
          print("No solutions")
          exit()
  def draw_map_2D(self,current_position):
    # Clear screen in Windows systems
    # os.system('cls')
    # Clear screen in Mac/Linux systems
    os.system('clear')
    print("-"*len(map[0])*4)
    for i in range(len(map)):
      print('|',end='')
      for j in range(len(map[0])):
          if current_position.get_state() == "STANDING" and i == current_position.i1 and j == current_position.j1:
              print("", "\033[1;34;47m x", "|", end='')
          elif (current_position.get_state() == "LYING_HORIZONTALLY" or current_position.get_state() == "LYING_VERTICALLY") and (i == current_position.i1 and j == current_position.j1) or (i == current_position.i2 and j == current_position.j2):
              print("", "\033[1;34;47m x", "|", end='')
          else:
              print("",map[i][j], "|", end='')
      print()
      print("-"*len(map[0])*4)
    
    time.sleep(1)

In [37]:
class GeneticAlgorithm(BLOXORZ):
  population = []
  prob = []
  def __init__(self, game, chromosome_length, population_capacity):
    self.game = game
    self.chromosome_length = chromosome_length
    self.population_capacity = population_capacity
    self.initPopulation()

  def initIndividual(self):
    chromosome = []
    for i in range(self.chromosome_length):
      # if chromosome is empty or having NONE as the last gen, add an arbitrary
      if len(chromosome) == 0 or chromosome[-1] == 0: chromosome.append(random.choice((0,1,2,3,4)))
      # if the last gen is UP, avoid DOWN
      elif chromosome[-1] == 1: chromosome.append(random.choice((0,1,3,4)))
      # if the last gen is DOWN, avoid UP
      elif chromosome[-1] == 2: chromosome.append(random.choice((0,2,3,4)))
      # if the last gen is LEFT, avoid RIGHT
      elif chromosome[-1] == 3: chromosome.append(random.choice((0,1,2,3)))
      # if the last gen is LEFT, avoid RIGHT
      else: chromosome.append(random.choice((0,1,2,4)))
    return chromosome
  
  def initPopulation(self):
    while len(self.population) < self.population_capacity:
      individual = self.initIndividual()
      final = self.game.move_process(individual)
      if final != -1: 
        self.population.append(individual)
        self.prob.append(self.game.cost_to_goal(final))

    _sum = sum(self.prob)
    for i in range(len(self.prob)): self.prob[i] = self.prob[i] / _sum;

In [44]:
initBlock = Position(3,1,3,1)
map = [[0,0,0,0,0,0,2,2,2,2,2,2,2,0,0],
        [2,2,2,2,0,0,2,2,2,0,0,2,2,0,0],
        [2,2,2,2,2,2,2,2,2,0,0,2,2,2,2],
        [2,2,2,2,0,0,0,0,0,0,0,2,2,10,2],
        [2,2,2,2,0,0,0,0,0,0,0,2,2,2,2],
        [0,0,0,0,0,0,0,0,0,0,0,0,2,2,2]]
game = BLOXORZ(map, initBlock)

game.move_process([1,2,0,3,2])

# for move in [1,2,0,3,2]:
#   initBlock = initBlock.move(move)
#   initBlock.print()

GA = GeneticAlgorithm(game, 5, 10)
print(GA.population)
print(GA.prob)



[[0, 1, 4, 0, 3], [4, 2, 0, 1, 3], [4, 1, 1, 3, 2], [0, 4, 0, 1, 4], [4, 0, 1, 4, 0], [4, 0, 1, 4, 0], [1, 3, 0, 2, 0], [1, 0, 3, 0, 0], [0, 1, 3, 2, 0], [1, 0, 3, 2, 4]]
[0.10526315789473684, 0.10526315789473684, 0.10526315789473684, 0.07894736842105263, 0.07894736842105263, 0.07894736842105263, 0.11403508771929824, 0.11403508771929824, 0.11403508771929824, 0.10526315789473684]


In [1]:
array = [[1,2,3],[4,5,6],[7,8,9]]
print([(index, row.index(5)) for index, row in enumerate(array) if 5 in row][0])

(1, 1)


In [ ]:
p1 = Position(1,2,3,4)
p2 = copy.copy(p1)

print(id(p1), id(p2))
p2.i1 = 2
print(p1.i1)
print(id(p1.i1), id(p2.i1))

140368414447888 140368414448592
1
11126688 11126720
